In [ ]:
%matplotlib inline
import numpy as np
import os
import time
import h5py
import keras
import pandas as pd
import math
import joblib
import json
import scipy
import matplotlib.pyplot as plt
from scipy.stats import logistic

from IPython.display import display

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from keras.layers import (Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, 
                          Activation, Dropout, Conv2D, Conv2DTranspose, LocallyConnected2D,
                          Concatenate, Add, Multiply)
from keras.engine import InputSpec
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import Model
from keras.utils import to_categorical
from keras.regularizers import l2
from keras import metrics
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from keras.datasets import cifar10

import tensorflow as tf

from pixelcnn_helpers import pixelcnn_loss, sigmoid, compute_pvals, compute_mixture


# Test scenarios for loss function

In [ ]:
img_rows, img_cols, img_chns = 1, 1, 3
n_components = 4
dummy_image = np.array([[[0.1, 0.1, 0.1]]])

def create_test(w):
    network_out = np.array([[[
        # M:r0 r1 g0 g1 b0 b1
        0, 0.5, 0.1, 0.2, 
        0, 0.5, 0.1, 0.2, 
        0, 0.5, 0.1, 0.2,
        # S:r0 r1 g0 g1 b0 b1
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        # W:r0 r1 g0 g1 b0 b1
        w[0], w[1], w[2], w[3], w[0], w[1], w[2], w[3], w[0], w[1], w[2], w[3],
    ]]])
    
    main_in = Input(shape=(img_rows, img_cols, img_chns), name='main_input')
    node = Dense(img_rows*img_cols*img_chns*n_components * 3, 
                 kernel_initializer='zeros', 
                 bias_initializer='zeros')(main_in)
    node = Reshape(network_out.shape)(node)
    main_out = Lambda(lambda x: x + K.constant(network_out))(node)

    optimizer = RMSprop(lr=0.001)
    model = Model(inputs=main_in, outputs=main_out)
    
    loss = lambda x, y: pixelcnn_loss(x, y, img_rows, img_cols, img_chns, n_components)
    
    model.compile(optimizer=optimizer, loss=loss)
    return model



In [ ]:
model = create_test([0.25, 0.25, 0.3, 0.2])

model.evaluate(x=[[dummy_image]], y=[[dummy_image]])

In [ ]:
model = create_test([0.01, 0.04, 0.5, 0.90])

model.evaluate(x=[[dummy_image]], y=[[dummy_image]])

In [ ]:
model = create_test([0.01, 0.99, 0., 0.])

model.evaluate(x=[[dummy_image]], y=[[dummy_image]])

In [ ]:
model = create_test([1e-12, 1-1e-12, 1e-12, 1e-12])

model.evaluate(x=[[dummy_image]], y=[[dummy_image]])

In [ ]:
model = create_test([0, 0., 1., 0.])

model.evaluate(x=[[dummy_image]], y=[[dummy_image]])